In [1]:
import re #regex for password criteria checking
import getpass #password input box
import json
from pathlib import Path
import os.path
import os
import hashlib
from Crypto.Cipher import AES

In [74]:
###########################################################
# Functions 
# This function takes in an email and password, and returns true/false based on successful authentication
def loginAuthentication(usersObj, emailAddress, inputPassword):
    passwordHash = hashPassword(inputPassword, usersObj.getPasswordHash(emailAddress)[:32])
    print('password hash')
    print(usersObj.getPasswordHash(emailAddress)[32:])
    print('password input hash')
    print(passwordHash[32:])
    print('is user registered')
    print(usersObj.isUserRegistered(emailAddress))
    print('do hashes equal?')
    print(usersObj.getPasswordHash(emailAddress)[32:] == passwordHash)
    if(usersObj.isUserRegistered(emailAddress) and usersObj.getPasswordHash(emailAddress)[32:] == passwordHash[32:]):
        return True #keeps the loop running until successful combination
    else:
        print('Incorrect email/password combination\n')
        return False #exits while loop
        print('\n')

# Takes a password and returns a sha-256 hashvalue with a salt
def hashPassword(password, salt) :
    # SHA hashing
    hashResult = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), salt, 200000) #200,000 iterations for SHA algorithm
    # Storing the hash and salt
    storage = salt + hashResult #[:32] is salt [32:] is key

    return salt + hashResult

def encryptContact(plaintext, passwordHash):
    session_key = passwordHash[32:]
    aes_cipher = AES.new(session_key, AES.MODE_EAX)
    ciphertext,tag = aes_cipher.encrypt_and_digest(plaintext)
    
    return [ciphertext, tag, aes_cipher.nonce]

def decryptContact(decryptArray, passwordHash):
    session_key = passwordHash
    nonce = decryptArray[2]
    tag = decryptArray[1]
    ciphertext = decryptArray[0]
    
    aes_cipher = AES.new(session_key, AES.MODE_EAX, nonce)
    plaintext = aes-cipher.decrypt_and_verify(ciphertext, tag)
    
    return plaintext

# New user generation, should generate a user and interface the file to save
def newUser():
    try_flag = True # used for password match checking

    fullName = input("Enter Full Name: ")
    email = input("Enter Email Address: ")
    while 1: # prompt the user until the passwords match
        if try_flag == False:
            # create junk values, user won't be created
            fullName = ""
            email = ""
            passWd = ""
            break
        passWd = input("Input Password: ")
        passWd_Check = input("Re-Enter Password: ")
        
        if passWd != passWd_Check:
            while 1: # prompt until accepted "try again?" answer
                try_again = input("Passwords do not match, try again? (y/n):")
                if (try_again == 'y' or try_again == 'Y'):
                    try_flag = True
                    break
                elif (try_again == 'n' or try_again == 'N'):
                    try_flag = False
                    break
                else: # invalid answer
                    print("User did not input y/n")
                
        else:
           # print("Passwords Match.")
            # Hashes the password using SHA-256
            # Use passWdHashed[32:] to get the key
            # Use passWdHashed[:32] to get the salt
            # 32 byte salt
            salt = os.urandom(32)
            passWdHashed = hashPassword(passWd, salt)
            break
    user = {
        "Name": fullName,
        "Email": email,
        "Password": passWdHashed
    }
    return user # return the user object to main to be saved

In [6]:



# data object that stores the user's 
class UserData:
    userData_file_path = Path(".SFTP_USERS.dat")
    temp_data_file_path = Path(".SFTP_USERS.tmp")
    data = {}
    enc_data = {}
    data_fp = None
    temp_fp = None
    
    def __init__(self,filePath=userData_file_path,tempFile=temp_data_file_path):
        if os.path.exists(filePath):
            print('')
            self.data_fp = open(filePath, "w+")
            self.temp_fp = open(tempFile, "w+")
            self.enc_data = json.load(self.data_fp)
        else:
            self.data = dict()
            self.data_fp = open(filePath, "w+")
            self.temp_fp = open(tempFile, "w+")
        return None
    
    def __del__(self):
        self.writeData()
        return None
    
    def isEmpty(self):
        return not(bool(self.data))
    
    def addUser(self, user):
        self.data[user['Email']] = user
        self.data[user['Email']]['Contacts'] = []
        self.enc_data[user['Email']] = self.data[user['Email']]
    
    def isUserRegistered(self, email):
        if email in self.enc_data.keys():
            return True
        else:
            return False
        
    def getPasswordHash(self, email):
        return self.enc_data[email]["Password"]
    
    def getContacts(self, email):
        if self.isUserRegistered(user):
            self.decrpytContacts(email, self.getPasswordHash(email))
            return self.data[email]['Contacts']
        else:
            return []
    
    def doesContactExist(self, email, contact):
        if self.isUserRegistered(email) and contact in self.data[email]['Contacts']:
            return True
        else:
            return False
    
    def addContact(self, email, contact):
        self.data[email]['Contacts'].append(contact)
        self.encryptContacts(email, self.getPasswordHash(email))
        return None
    
    def removeContact(self, email, contact):
        if self.doesContactExist(email, contact):
            self.data[email]['Contacts'].remove(contact)
            self.encryptContacts(email, self,getPasswordHash(email))
            return True
        else:
            return False
    
    def loadData(self):
        # TODO:
        return None
    
    def writeData(self):
        try:
            print(json.dumps(self.enc_data))
            json.dump(self.enc_data, self.temp_fp)
            self.temp_fp.close()
            self.data_fp.close()
            os.remove(userData_file_path)
            os.rename(temp_data_file_path, userData_file_path)
        except OSError:
            return False
        return True
    
    def decryptContacts(self, email, passwordHash):
        if self.data[email]['Password'] == passwordHash:
            self.data[email] = decryptContact(passwordHash, json.loads(self.enc_data[email]['Contacts']))
            return True
        else:
            return False
        
    def encryptContacts(self, email):
        self.enc_data[email]['Contacts'] = encryptContact(json.dumps(self.data[email]['Contacts']), self.data[email]['Password'])
        return None







In [7]:
###########################################################
# Start of main program
usersObj = UserData()
userList = not usersObj.isEmpty() # If there are no users in file 
#samuel insert code around here for file checking

#If there are no users in file, create initial user in while loop
while (userList == False):
    print('No users are registered with this client')
    firstUser= input('Would you like to register a new user now? (y/n): ')
    if (firstUser == 'Y' or firstUser == 'y'):
        
        # create the new user
        user = newUser()
        if (user["Name"] == "" or user["Email"] == "" or user["Password"] == ""): # was a contact unable to be created
            print('You cannot use this system without a user registered')
        else: # user created, now store it 
            usersObj.addUser(user)
             
        
        userList = True #can be changed with user registration code (must not == False to exit loop)
    else:
        print('You cannot use this system without a user registered')

print('\n$Welcome to Secure Drop!$ \n--- Login ---')

loginSuccess = False # Exits the while loop when not False
while (loginSuccess == False):
    emailAddress = input('Email Address:')
    inputPassword = input('Password:')
    loginSuccess = loginAuthentication(usersObj, emailAddress, inputPassword) # Changes the boolean to True if successful
print("Login was successful!")

exit_shell = False # Exits the while loop when not False
while (exit_shell == False):
    # 'shell' prefix for each line
    textInput = input('$secure_drop>')
    
    # Add command
    if(textInput == 'add'):
        # Adds users to the contact list for the logged in user
        print('\n--- Add Contact ---')
        contact_name = input('Enter Full Name: ')
        contact_email = input('Enter Email Address: ')
        
        # create the contact dict
        contact = {
            "Email": contact_email,
            "Name": contact_name
        }
        # add contact to user 
        usersObj.addContact(emailAddress, contact)
        
        # was the contact properly stored?
        if(usersObj.doesContactExist(emailAddress, contact)): #needs to check our implementation of user storage
            print('Contacted added to List')
        else:
            print('Failed to add contact')
            
    # List command
    elif(textInput == 'list'):
        #molly/sammy code to pull the list of users from the file
        print('test')
        
        
    # Send command
    elif(textInput == 'send'):
        print('test2')
        
    
    # Help command
    elif(textInput == 'help'):
        print('\t"add" -> Add a new contact \n\t\"list\" -> List all online contacts \n\t\"send\" -> Transfer file to contact \n\t\"exit\" -> Exit SecureDrop')
        
    # Exit command
    elif(textInput == 'exit'):
        exit_shell= True
        
print('Exiting Secure Drop')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
usersObj.__del__()